<a href="https://colab.research.google.com/github/mbilalnust/spam_sms_text_classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the train and test data
train_df = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

# Optional: explore
print(train_df.head())
print(train_df['label'].value_counts())

In [ ]:
# Encode labels: ham = 0, spam = 1
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

In [ ]:
print(train_df.head())
print(train_df['label'].value_counts())

In [ ]:
# ---- [3. OVERSAMPLING TO BALANCE CLASSES] ----
ham_df = train_df[train_df['label'] == 0]
spam_df = train_df[train_df['label'] == 1]

spam_oversampled = spam_df.sample(len(ham_df), replace=True, random_state=42)
train_df_balanced = pd.concat([ham_df, spam_oversampled]).sample(frac=1, random_state=42)

print(train_df_balanced['label'].value_counts())


In [ ]:
# ---- [4. TEXT VECTORIZATION] ----
max_features = 10000
sequence_length = 100

vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)
vectorizer.adapt(train_df_balanced['message'].values)

In [ ]:
# ---- [5. CREATE DATASETS] ----
def df_to_dataset(df):
    messages = df["message"].values
    labels = df["label"].values
    return tf.data.Dataset.from_tensor_slices((messages, labels))

train_ds = df_to_dataset(train_df_balanced).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = df_to_dataset(test_df).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
# ---- [6. BUILD THE MODEL] ----
model = tf.keras.Sequential([
    vectorizer,
    tf.keras.layers.Embedding(max_features, 64, mask_zero=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
# ---- [7. TRAIN THE MODEL] ----
model.fit(train_ds, validation_data=test_ds, epochs=10)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Create a TensorFlow constant of dtype string with batch size 1
    input_tensor = tf.constant([pred_text], dtype=tf.string)
    prob = model.predict(input_tensor)[0][0]
    label = "spam" if prob > 0.5 else "ham"
    return [float(prob), label]


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
